# Modelo de Generaciones Traslapadas Dinámico y Estocástico

Este notebook es una guía para la instalación y ejecución básica del Modelo de Generaciones Traslapadas Dinámico y Estocástico realizado para el BID bajo el proyecto “A general equilibrium model for a new social contract: health, pensions, and care”.

Este modelo incorpora a riesgos idiosincráticos en la productividad laboral y decisión endógena de la oferta de laboral de los hogares. El precio de los factores responde a cambios en el comportamiento individual y el gobierno entra como un agente que recolecta ingresos por impuestos para financiar su gasto.

In [1]:
# Descargamos las funciones auxiliares para correr el modelo
using Downloads 

utils_url = "https://raw.githubusercontent.com/milocortes/olg_egtp/main/src/julia/programas/ch11/prog11_02/dev/no_fortran/utils_surv_prob_no_fortran.jl"
utils_local_path = joinpath(pwd(),"utils_dsolg.jl")

Downloads.download(utils_url,utils_local_path)

# Cargamos las funciones auxiliares
include(utils_local_path)

check_grid (generic function with 1 method)

In [2]:
# Cargamos paqueterías
using OffsetArrays
using Roots
using DataFrames

# Número de periodos de transición
global TT = 40

# Número de años que viven los hogares
global JJ = 16

# Número de años a los que se retiran los hogares
global JR = 10

# Número de shocks persistentes
global NP = 2

# Número de shocks transitorios
global NS = 5

# Número de puntos del grid de activos
global NA = 100

# Parámetros de las preferencias de los hogares
global gamma = 0.50
global egam = 1.0 - 1.0/gamma
global nu    = 0.335
global beta  = 0.998^5

# Parámetros de la varianza en la formación de ingresos laborales
global sigma_theta = 0.23
global sigma_eps   = 0.05
global rho         = 0.98

# Parámetros de la producción
global alpha = 0.36
global delta = 1.0-(1.0-0.0823)^5
global Omega2 = 1.60

# Tamaño del grid de activos
global a_l    = 0.0
global a_u    = 50.0
global a_grow = 0.05

# Parámetros demográficos
global n_p   = (1.0+0.02)^5-1.0

# Parámetros de la simulación
global damp    = 0.30
global sig     = 1e-4
global itermax = 70


70

In [3]:
# DEFINICIÓN DE ARREGLOS QUE ALMACENARÁN LAS SALIDAS DEL MODELO

## Variables macroeconómicas
### Precios 
for param = [:r, :rn, :w, :wn, :p]
    @eval global $param = OffsetArray(zeros(TT+1), 0:TT);
end

### Mercado de Capital
for param = [:KK, :AA, :BB, :LL, :HH]
    @eval global $param = OffsetArray(zeros(TT+1), 0:TT)
end

### Mercado de Bienes
for param = [:YY, :CC, :II, :GG, :INC, :BQ]
    @eval global $param = OffsetArray(zeros(TT+1), 0:TT)
end

### Variables del Gobierno
for param = [:tauc, :tauw, :taur, :taup, :kappa, :PP]
    @eval global $param = OffsetArray(zeros(TT+1), 0:TT)
end

global gy 
global by 

global pen = OffsetArray(zeros(JJ,TT+1), 1:JJ,0:TT)

global taxrev = OffsetArray(zeros(4,TT+1), 1:4,0:TT)
global tax = OffsetArray(Int.(zeros(TT+1)), 0:TT)

### Variables para calcular LSRA 
global BA = OffsetArray(zeros(TT+1), 0:TT)
global SV = OffsetArray(zeros(TT+1), 0:TT) 

global lsra_comp
global lsra_all
global Lstar
global lsra_on
global Vstar

### Variables para agregación a nivel de cohorte 
for param = [:c_coh, :y_coh, :l_coh, :a_coh, :v_coh, :VV_coh]
    @eval global $param = OffsetArray(zeros(JJ, TT+1), 1:JJ, 0:TT) ;
end

for param = [:GAM, :beq, :beq_coh]
    @eval global $param = OffsetArray(zeros(JJ, TT+1), 1:JJ, 0:TT) ;
end

global omega = zeros(JJ)

global psi = OffsetArray(zeros(JJ+1, TT+1),1:JJ+1, 0:TT)


# Variables que almacenan el proceso de shocks de productividad
global dist_theta = zeros(NP)
global theta = zeros(NP)
global is_initial = 3

# Parámetros demográficos y de eficiencia laboral
global eff = zeros(JJ)

for param = [:m, :pop]
    @eval global $param = OffsetArray(zeros(JJ, TT+1), 1:JJ, 0:TT) ;
end

# Variables individuales

global a = OffsetArray(zeros(NA+1),0:NA)

global aplus = OffsetArray(zeros(JJ, NA+1, NP, NS, TT+1), 1:JJ, 0:NA, 1:NP, 1:NS, 0:TT)

for param = [:c, :l, :phi, :VV, :v]
    @eval global $param = OffsetArray(zeros(JJ, NA+1, NP, NS, TT+1), 1:JJ, 0:NA, 1:NP, 1:NS, 0:TT)
end 

global FLC = OffsetArray(zeros(JJ,TT+1), 1:JJ,0:TT)

# Variables numéricas
global RHS = OffsetArray(zeros(JJ, NA+1, NP, NS, TT+1), 1:JJ, 0:NA, 1:NP, 1:NS, 0:TT) 
global EV = OffsetArray(zeros(JJ, NA+1, NP, NS, TT+1), 1:JJ, 0:NA, 1:NP, 1:NS, 0:TT) 

for param = [:ij_com, :ia_com, :ip_com, :is_com, :it_com, :cons_com, :lab_com]
    @eval global $param
end

global DIFF = OffsetArray(zeros(TT+1),0:TT)



41-element OffsetArray(::Vector{Float64}, 0:40) with eltype Float64 with indices 0:40:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [4]:


##### Inicializamos los parámetros y variables restantes del modelo


# Probabilidades de sobrevivencia por cohorte
psi[1:6,0:TT] .= 1.00000000
psi[7,0:TT] .= 0.98972953
psi[8,0:TT] .= 0.98185396
psi[9,0:TT] .= 0.97070373
psi[10,0:TT] .= 0.95530594
psi[11,0:TT] .= 0.93417914
psi[12,0:TT] .= 0.90238714
psi[13,0:TT] .= 0.83653436
psi[14,0:TT] .= 0.71048182
psi[15,0:TT] .= 0.52669353
psi[16,0:TT] .= 0.31179803
psi[17,0:TT] .= 0.00000000

# Distribución de las herencias
omega[1] = 1.0/6.0
omega[2] = 1.0/6.0
omega[3] = 1.0/6.0
omega[4] = 1.0/6.0
omega[5] = 1.0/6.0
omega[6] = 1.0/6.0
#        omega(7) = 1d0/9d0
#        omega(8) = 1d0/9d0
#        omega(9) = 1d0/9d0
omega[7:16] .= 0.0

# Configuramos la estructura poblacional 
for it in 0:TT
    m[1,it] = 1.0
    GAM[1,it] = omega[1]
    itm = year2(it, -1)
    for ij in 2:JJ
        m[ij,it] = m[ij-1, itm]*psi[ij, it]/(1.0+n_p)
        GAM[1,it] = GAM[1, it] + omega[ij]*m[ij, it]
    end
    for ij in JJ:-1:1
        GAM[ij, it] = omega[ij]/GAM[1, it]
    end
end

# Inicializamos el grid de activos
grid_Cons_Grow(a, NA+1, a_l, a_u, a_grow)

In [5]:


# Obenemos la conjetura inicial para las decisiones de ahorro
for ij in 1:JJ
    for ip in 1:NP
        for is in 1:NS
            aplus[ij, :, ip, is, 0] .= max.(a/2, a[1]/2) 
        end
    end
end

# Inicializamos el perfil de eficiencia de gananacias laborales por cohorte 
eff[1] = 1.0000
eff[2] = 1.3527
eff[3] = 1.6952
eff[4] = 1.8279
eff[5] = 1.9606
eff[6] = 1.9692
eff[7] = 1.9692
eff[8] = 1.9392
eff[9] = 1.9007
eff[JR:JJ] .= 0.0

# Inicializamos el efecto fijo de productividad laboral
dist_theta .= 1.0/float(NP)
theta[1]   = -sqrt(sigma_theta)
theta[2]   = sqrt(sigma_theta)
theta .= exp.(theta)

# Obtenemos la discretización de los shocks aleatorios mediante el algoritmo de : Rouwenhorst
pi, eta = rouwenhorst(NS, rho, sigma_eps, 0.0);
eta = exp.(eta)

# Inicializamos esquemas de impuestos y transferencias
tax   .= 2
tauc  .= 0.075
tauw  .= 0.0
taur  .= 0.0
taup  .= 0.1
kappa .= 0.5
gy    = 0.19
by    = 0.60/5.0

beq[:,0] .= 0.0
BQ[0] = 0.0

# Conjeturas iniciales para las variables macroeconómicas
KK .= 1.0
LL .= 1.0
YY .= 1.0
II .= (n_p+delta)*KK

GG .= gy*YY[0]
BB .= by*YY[0]

pen .= 0.0
pen[JR:JJ, 0] .= kappa[0]


global ial
global iar
global varphi


In [6]:
# Calculamos el equilibrio inicial
get_SteadyState()

INITIAL EQUILIBRIUM
ITER     H     K/Y     C/Y     I/Y       r       w        DIFF
1     0.34   404.87   133.06   36.7   4.17   1.02   -74.799094
2     0.36   364.27   84.07   33.02   1.84   1.18   -31.356449
3     0.34   336.75   68.64   30.52   2.75   1.12   -15.936406
4     0.34   324.49   61.62   29.41   3.46   1.07   -8.660748
5     0.33   319.18   57.98   28.93   3.81   1.05   -5.02016
6     0.33   316.26   55.97   28.67   3.97   1.04   -3.062849
7     0.33   314.37   54.77   28.49   4.06   1.03   -1.910052
8     0.33   313.15   54.04   28.38   4.12   1.03   -1.19908
9     0.33   312.38   53.59   28.31   4.15   1.03   -0.754677
10     0.32   311.9   53.3   28.27   4.18   1.02   -0.475504
11     0.32   311.6   53.12   28.24   4.19   1.02   -0.299534
12     0.32   311.41   53.0   28.23   4.2   1.02   -0.188339
13     0.32   311.29   52.93   28.22   4.21   1.02   -0.118055
14     0.32   311.22   52.88   28.21   4.21   1.02   -0.073698
15     0.32   311.17   52.85   28.2   4.21   1.0

In [7]:
# Recolectamos resultados

capital_market = DataFrame( 
          etiqueta = ["valor", "(in %)"],
          K = [KK[0], KK[0]/YY[0]*500],
          A = [AA[0], AA[0]/YY[0]*500],  
          B = [BB[0], BB[0]/YY[0]*500], 
          BA = [BA[0], BA[0]/YY[0]*500],   
          r = [r[0], ""],
          pa = [((1.0+r[0])^(1.0/5.0)-1.0)*100.0,  ""]
);

labour_market = DataFrame( 
    etiqueta = ["valor"],
    L = [LL[0]],
    HH = [HH[0]*100],
    INC = [INC[0]],
    w = [w[0]]
);

good_market = DataFrame( 
          etiqueta = ["valor", "(in %)"],
          Y = [YY[0], YY[0]/YY[0]*100],
          C = [CC[0], CC[0]/YY[0]*100],  
          I = [II[0], II[0]/YY[0]*100], 
          G = [GG[0], GG[0]/YY[0]*100]
); 

gov_accounts = DataFrame(
    etiqueta = ["valor", "(in %)", "(rate)"],
    TAUC = [taxrev[1,0], taxrev[1,0]/YY[0]*100, tauc[0]*100],   
    TAUW = [taxrev[2,0], taxrev[2,0]/YY[0]*100, tauw[0]*100],
    TAUR = [taxrev[3,0], taxrev[3,0]/YY[0]*100, taur[0]*100],
    TOTAL = [taxrev[4,0], taxrev[4,0]/YY[0]*100, ""], 
    G = [GG[0], GG[0]/YY[0]*100, ""],
    B = [BB[0], (BB[0]*5.0)/YY[0]*100, ""]
);

pension_system = DataFrame(
    etiqueta = ["valor", "(in %)"],
    TAUP = [taup[0]*w[0]*LL[0], taup[0]*100], 
    PEN = [pen[JR, 0], kappa[0]],
    PP = [PP[0], PP[0]/YY[0]*100]
);

In [8]:
# MERCADO DE CAPITALES
capital_market

Row,etiqueta,K,A,B,BA,r,pa
,String,Float64,Float64,Float64,Float64,Any,Any
1,valor,4.3429,5.18051,0.83758,0.0,0.229469,4.21819
2,(in %),311.104,371.105,60.0,0.0,,


In [9]:
# MERCADO DE TRABAJO
labour_market


Row,etiqueta,L,HH,INC,w
,String,Float64,Float64,Float64,Float64
1,valor,4.3734,32.334,0.719209,1.02142


In [10]:
# MERCADO DE BIENES
good_market

Row,etiqueta,Y,C,I,G
,String,Float64,Float64,Float64,Float64
1,valor,6.97983,3.68548,1.96819,1.32617
2,(in %),100.0,52.8019,28.1982,19.0


In [11]:
# GOBIERNO
gov_accounts

Row,etiqueta,TAUC,TAUW,TAUR,TOTAL,G,B
,String,Float64,Float64,Float64,Any,Any,Any
1,valor,0.276411,0.912064,0.242715,1.43119,1.32617,0.83758
2,(in %),3.96014,13.0671,3.47738,20.5047,19.0,60.0
3,(rate),7.5,20.4174,20.4174,,,


In [12]:
# SISTEMA DE PENSIONES
pension_system

Row,etiqueta,TAUP,PEN,PP
,String,Float64,Float64,Float64
1,valor,0.479208,0.359604,0.479208
2,(in %),10.7275,0.5,6.86561


In [16]:
# Definimos un cambio en los parámetros iniciales para modelar una reforma
kappa[1:TT] .= 0.5;

global sig     = 1e-4
# Calculamos la trayectoria de transición al nuevo estado estacionario sin LRSA
lsra_on = false;

get_transition()

TRANSITION PATH
ITER       H     K/Y     C/Y     I/Y       r       w        DIFF
1     0.32   311.1   52.8   28.2   4.22   1.02   9.0e-5
2     0.32   311.1   52.8   28.2   4.22   1.02   0.00012
3     0.32   311.1   52.8   28.2   4.22   1.02   0.0001
4     0.32   311.1   52.8   28.2   4.22   1.02   8.0e-5


In [22]:
#### GUARDAMOS LOS RESULTADOS

macro Name(arg)
    string(arg)
end

## Variables agregadas

results_1D_df = []

for param = [:r, :rn, :w, :wn, :p, :KK, :AA, :BB, :LL, :HH, :YY, :CC, :II, :GG, :INC, :BQ, :tauc, :tauw, :taur, :taup, :kappa, :PP]
    push!(results_1D_df,
        @eval DataFrame($param = [$param...])  
    )
end

results_1D_df = hcat(results_1D_df...)

## Variables por cohorte

results_2D_df = []

for param = [:pen, :c_coh, :y_coh, :l_coh, :a_coh, :v_coh, :VV_coh]
    @eval col_names = [string(@Name($param),"_cohort_",i) for i in 1:size($param.parent)[1]]
    push!(results_2D_df,
        @eval DataFrame(OffsetArrays.no_offset_view($param)', col_names)
    )
end

results_2D_df = hcat(results_2D_df...)

## Recaudación de impuestos
df_tax_rev = DataFrame(OffsetArrays.no_offset_view(taxrev)', ["tauc_rev", "tauw_rev", "taur_rev", "total_tax_rev"])

## Recolectamos los datos
results_all = hcat([ DataFrame(time = 0:TT), results_1D_df, results_2D_df, df_tax_rev]...)


Row,time,r,rn,w,wn,p,KK,AA,BB,LL,HH,YY,CC,II,GG,INC,BQ,tauc,tauw,taur,taup,kappa,PP,pen_cohort_1,pen_cohort_2,pen_cohort_3,pen_cohort_4,pen_cohort_5,pen_cohort_6,pen_cohort_7,pen_cohort_8,pen_cohort_9,pen_cohort_10,pen_cohort_11,pen_cohort_12,pen_cohort_13,pen_cohort_14,pen_cohort_15,pen_cohort_16,c_coh_cohort_1,c_coh_cohort_2,c_coh_cohort_3,c_coh_cohort_4,c_coh_cohort_5,c_coh_cohort_6,c_coh_cohort_7,c_coh_cohort_8,c_coh_cohort_9,c_coh_cohort_10,c_coh_cohort_11,c_coh_cohort_12,c_coh_cohort_13,c_coh_cohort_14,c_coh_cohort_15,c_coh_cohort_16,y_coh_cohort_1,y_coh_cohort_2,y_coh_cohort_3,y_coh_cohort_4,y_coh_cohort_5,y_coh_cohort_6,y_coh_cohort_7,y_coh_cohort_8,y_coh_cohort_9,y_coh_cohort_10,y_coh_cohort_11,y_coh_cohort_12,y_coh_cohort_13,y_coh_cohort_14,y_coh_cohort_15,y_coh_cohort_16,l_coh_cohort_1,l_coh_cohort_2,l_coh_cohort_3,l_coh_cohort_4,l_coh_cohort_5,l_coh_cohort_6,l_coh_cohort_7,l_coh_cohort_8,l_coh_cohort_9,l_coh_cohort_10,l_coh_cohort_11,l_coh_cohort_12,l_coh_cohort_13,l_coh_cohort_14,l_coh_cohort_15,l_coh_cohort_16,a_coh_cohort_1,a_coh_cohort_2,a_coh_cohort_3,a_coh_cohort_4,a_coh_cohort_5,a_coh_cohort_6,a_coh_cohort_7,a_coh_cohort_8,a_coh_cohort_9,a_coh_cohort_10,a_coh_cohort_11,a_coh_cohort_12,a_coh_cohort_13,⋯
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0,0.229469,0.182618,1.02142,0.703303,1.075,4.3429,5.18051,0.83758,4.3734,0.32334,6.97983,3.68548,1.96819,1.32617,0.719209,0.208813,0.075,0.204174,0.204174,0.107275,0.5,0.479208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.359604,0.359604,0.359604,0.359604,0.359604,0.359604,0.359604,0.2499,0.311122,0.376413,0.430852,0.489782,0.54174,0.593065,0.641766,0.687697,0.680846,0.728144,0.758791,0.7471,0.654294,0.500637,0.356237,0.358977,0.609481,0.856211,0.90149,0.93943,0.848247,0.752913,0.63372,0.518821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.310363,0.379296,0.413081,0.388154,0.361494,0.306246,0.253771,0.198269,0.145558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0277699,0.170978,0.443675,0.739495,1.05267,1.30305,1.43299,1.45048,1.34098,1.21356,1.01202,0.740741,⋯
2,1,0.229469,0.182618,1.02142,0.703303,1.075,4.34292,5.18051,0.83758,4.37342,0.323339,6.97986,3.68549,1.9682,1.32617,0.719212,0.208813,0.075,0.204173,0.204173,0.107275,0.5,0.479208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.359604,0.359604,0.359604,0.359604,0.359604,0.359604,0.359604,0.249901,0.311122,0.376414,0.430853,0.489783,0.541741,0.593065,0.641767,0.687698,0.680847,0.728146,0.758793,0.747101,0.654296,0.500638,0.356238,0.358976,0.60948,0.85621,0.901488,0.939429,0.848246,0.752913,0.633719,0.51882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.310361,0.379295,0.413079,0.388153,0.361493,0.306245,0.253771,0.198268,0.145558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0277699,0.170978,0.443675,0.739495,1.05267,1.30305,1.43299,1.45048,1.34098,1.21356,1.01202,0.740741,⋯
3,2,0.229469,0.182618,1.02142,0.703303,1.075,4.34292,5.18051,0.83758,4.37341,0.323339,6.97986,3.68549,1.9682,1.32617,0.719212,0.208813,0.075,0.204173,0.204173,0.107275,0.5,0.47921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.359606,0.359606,0.359606,0.359606,0.359606,0.359606,0.359606,0.249901,0.311122,0.376414,0.430853,0.489783,0.541741,0.593065,0.641767,0.687698,0.680847,0.728146,0.758793,0.747102,0.654296,0.500639,0.356239,0.358975,0.609479,0.856209,0.901488,0.939428,0.8